# Data Analytics (Spark SQL)

This notebook runs **Analytics queries** in Spark SQL.

- Explores dimensions, measures, rankings, and trends
- Performs cumulative and segmentation analysis
- Evaluates performance and part-to-whole contributions

The notebook demonstrates how to query the Gold layer for insights.
